In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
from programa_geovars import *
import matplotlib.pyplot as plt

In [3]:
df_comercios = gpd.read_file('./nomecalles2/Colectivo empresarial por tamaño y actividad/dir2019a.shp')

In [4]:
cod_postales.head()

,cartodb_id,codbdt,geocodigo,desbdt,geometry
0,10,828922,28010,28010,"POLYGON ((-3.69780 40.43820, -3.69710 40.43807..."
1,21,828933,28021,28021,"POLYGON ((-3.70598 40.34995, -3.70596 40.34975..."
2,46,828958,28046,28046,"POLYGON ((-3.68066 40.48248, -3.68009 40.48199..."
3,48,828960,28048,28048,"POLYGON ((-3.80935 40.60950, -3.80927 40.60941..."
4,49,828961,28049,28049,"POLYGON ((-3.68220 40.55254, -3.68228 40.55218..."


In [5]:
df_comercios.head()

,CMUN,CODR28,ESTRA_6,ETRS89_X,ETRS89_Y,ETIQUETA,DIRECCIO,geometry
0,0066,15,4,444773,4486417,"SUPERMERCADOS CHAMPION, S.A.","crnacional i, km., 14,500 00000",POINT (444773.000 4486417.000)
1,0014,25,3,447506,4548576,Consultorio de La Acebeda,"Plaza de San Miguel, 1",POINT (447506.000 4548576.000)
2,0014,12,2,448297,4548183,"TECNOLOGIAS DEL MEDIO AMBIENTE, S.A.",Ctra de la Acebeda,POINT (448297.000 4548183.000)
3,0029,25,3,459130,4487430,Consultorio de Ajalvir,"Avda de las Aguas, 2",POINT (459130.000 4487430.000)
4,0029,12,2,459203,4487561,MORENO FRAGA RAMON,"Avda de las Aguas, 28",POINT (459203.000 4487561.000)


In [7]:
type(df_comercios.geometry.iloc[0])

shapely.geometry.point.Point

In [8]:
def get_postal_codes(points):
    '''
    Function to retrieve the postal code for points, where points are in the form (lon, lat).
    Using the cod_postales df, which has the polygons for each postal code, it checks which
    of those polygons each point falls into.

    Parameters
    ------------
    points
        a list or array-like with the points (lon, lat)
    
    Returns
    ------------
    The postal code for each of those points. It excludes the zeroes as they mean the postal code
    for that point could not be retrieved.
    '''
    codigos = np.zeros((len(points),))
    for i, p in tqdm(enumerate(points)):
        #if type(p)
        p = Point(p[0], p[1])
        for j in range(cod_postales.shape[0]):
            if cod_postales.geometry.iloc[j].contains(p):
                codigos[i] = cod_postales.geocodigo.iloc[j]
    return codigos[codigos!=0]

In [9]:
#pcodes = get_postal_codes(df_comercios.geometry)

In [10]:
points = get_lon_lat(df_comercios, nombre='comercios')

109720it [00:57, 1914.17it/s]


In [13]:
points = [(lon, lat) for lon, lat in zip(points['comercios']['lon'], points['comercios']['lat'])]

In [14]:
pcodes = get_postal_codes(points)

109720it [11:54, 153.51it/s]


In [15]:
df_comercios['CODIGO_POSTAL'] = pcodes

In [36]:
grouped_df = df_comercios.groupby(['CODIGO_POSTAL', 'ESTRA_6'])['ESTRA_6'].count()

In [54]:
gr = grouped_df.unstack()

In [62]:
cod_postales.columns = ['cartodb_id', 'codbdt', 'CODIGO_POSTAL', 'desbdt', 'geometry']

In [64]:
cod_postales.CODIGO_POSTAL = cod_postales.CODIGO_POSTAL.astype('float')

In [66]:
#pd.merge(gr, cod_postales, on='CODIGO_POSTAL')

In [60]:
gr.columns

Index(['2', '3', '4', '5', '6'], dtype='object', name='ESTRA_6')

In [61]:
gr.index

Float64Index([28001.0, 28002.0, 28003.0, 28004.0, 28005.0, 28006.0, 28007.0,
              28008.0, 28009.0, 28010.0,
              ...
              28976.0, 28977.0, 28978.0, 28979.0, 28981.0, 28982.0, 28983.0,
              28984.0, 28990.0, 28991.0],
             dtype='float64', name='CODIGO_POSTAL', length=295)

In [44]:
df = pd.DataFrame(df_comercios.groupby(['CODIGO_POSTAL', 'ESTRA_6'])['ESTRA_6'].count())

In [45]:
df.head()

ESTRA_6
CODIGO_POSTAL ESTRA_6         
28001.0       2           1876
              3            861
              4            120
              5             37
              6             20

In [51]:
df.unstack()

ESTRA_6                          
ESTRA_6             2      3      4     5     6
CODIGO_POSTAL                                  
28001.0        1876.0  861.0  120.0  37.0  20.0
28002.0        1222.0  471.0   66.0  34.0  20.0
28003.0        1086.0  465.0   49.0  34.0  23.0
28004.0        1494.0  684.0   61.0  32.0  10.0
28005.0         901.0  311.0   44.0  12.0  11.0
28006.0        1770.0  786.0   80.0  44.0  15.0
28007.0         889.0  317.0   48.0  33.0  19.0
28008.0         865.0  375.0   39.0  26.0   8.0
28009.0        1012.0  371.0   33.0  14.0  21.0
28010.0        1495.0  604.0   89.0  40.0  17.0
28011.0         415.0  139.0   22.0  10.0   3.0
28012.0        1020.0  372.0   38.0   9.0   4.0
28013.0         942.0  606.0   67.0  24.0  18.0
28014.0         830.0  398.0   63.0  29.0  32.0
28015.0        1297.0  373.0   53.0  24.0   8.0
28016.0         658.0  296.0   52.0  32.0   5.0
28017.0         677.0  165.0   30.0   6.0   NaN
28018.0         404.0  103.0   23.0   8.0   3.0
28019.0         676.0  181.0   30.0  13.0   1.0
28020.0        1621.0  812.0  104.0  88.0  39.0
28021.0         920.0  410.0   58.0  31.0  12.0
28022.0         633.0  347.0   71.0  18.0  21.0
28023.0         532.0  296.0   47.0  32.0  13.0
28024.0         299.0   98.0   13.0   3.0   1.0
28025.0         690.0  166.0   25.0   6.0   2.0
28026.0         677.0  228.0   40.0   8.0   6.0
28027.0        1029.0  398.0  101.0  41.0  32.0
28028.0         901.0  326.0   49.0  21.0  21.0
28029.0         668.0  245.0   31.0  11.0   9.0
28030.0         508.0  158.0   33.0  10.0   5.0
28031.0         739.0  350.0   50.0  30.0  19.0
28032.0         403.0  142.0   26.0  11.0   4.0
28033.0         673.0  301.0   85.0  52.0  31.0
28034.0         587.0  280.0   59.0  44.0  20.0
28035.0         514.0  189.0   28.0  15.0   7.0
28036.0         845.0  441.0   63.0  38.0  24.0
28037.0         706.0  457.0   98.0  72.0  46.0
28038.0         488.0  144.0   38.0   8.0   4.0
28039.0         557.0  143.0   32.0  10.0   3.0
28040.0         148.0  102.0   22.0  30.0  44.0
28041.0         533.0  241.0   64.0  19.0  13.0
28042.0         531.0  405.0  100.0  75.0  42.0
28043.0         952.0  356.0   69.0  47.0  25.0
28044.0         503.0  196.0   35.0  20.0   5.0
28045.0        1008.0  428.0   71.0  41.0  31.0
28046.0         978.0  687.0  111.0  87.0  47.0
28047.0         384.0  100.0   23.0   6.0   5.0
28048.0          23.0   11.0    NaN   1.0   NaN
28049.0         175.0  101.0   32.0  11.0  17.0
28050.0         598.0  369.0  102.0  78.0  51.0
28051.0         174.0  126.0   18.0   9.0   8.0
28052.0          46.0   54.0   13.0   2.0   2.0
28053.0         465.0  190.0   37.0  14.0   6.0
28054.0         181.0  124.0   21.0   8.0   4.0
28055.0          63.0   26.0    9.0   1.0   NaN
28100.0         591.0  249.0   36.0  19.0  10.0
28108.0         580.0  583.0  125.0  75.0  29.0
28109.0         269.0  185.0   37.0  33.0   8.0
28110.0          61.0   35.0    5.0   NaN   NaN
28119.0         112.0   79.0   16.0   4.0   1.0
28120.0          26.0   11.0    2.0   NaN   NaN
28130.0          25.0   13.0    2.0   1.0   NaN
28140.0          69.0   31.0    3.0   1.0   NaN
28150.0          26.0   18.0    NaN   NaN   NaN
28160.0          15.0    8.0    2.0   1.0   NaN
28170.0           6.0    1.0    NaN   NaN   NaN
28180.0          43.0   13.0    2.0   NaN   NaN
28189.0          28.0    7.0    NaN   NaN   1.0
28190.0           6.0    4.0    NaN   NaN   NaN
28191.0           7.0    6.0    NaN   NaN   NaN
28192.0           4.0    7.0    1.0   NaN   NaN
28193.0           2.0    NaN    NaN   NaN   NaN
28194.0           3.0    4.0    NaN   NaN   NaN
28195.0           NaN    1.0    NaN   NaN   NaN
28196.0           NaN    1.0    NaN   NaN   NaN
28200.0         124.0   53.0   10.0   6.0   3.0
28210.0          89.0   26.0    2.0   1.0   NaN
28211.0           2.0    NaN    NaN   NaN   NaN
28212.0           9.0    7.0    NaN   NaN   NaN
28213.0          16.0    9.0    NaN   NaN   1.0
28214.0           9.0    5.0  

In [47]:
df.index

MultiIndex([(28001.0, '2'),
            (28001.0, '3'),
            (28001.0, '4'),
            (28001.0, '5'),
            (28001.0, '6'),
            (28002.0, '2'),
            (28002.0, '3'),
            (28002.0, '4'),
            (28002.0, '5'),
            (28002.0, '6'),
            ...
            (28984.0, '2'),
            (28984.0, '3'),
            (28984.0, '5'),
            (28990.0, '2'),
            (28990.0, '3'),
            (28991.0, '2'),
            (28991.0, '3'),
            (28991.0, '4'),
            (28991.0, '5'),
            (28991.0, '6')],
           names=['CODIGO_POSTAL', 'ESTRA_6'], length=1105)

In [71]:

df_comercios.loc[['ZARA' in etiqueta for etiqueta in df_comercios.ETIQUETA], 'ETIQUETA']

833                                 MUZARAKE EVENTOS SL
851                         ZARA ESPAÑA, S.A. (INDITEX)
1401                     LUZARA SERVICIOS INTEGRALES SL
1589                          ZARAZAGA FRESNILLO CARLOS
2848                        ZARA ESPAÑA, S.A. (INDITEX)
2849                        ZARA ESPAÑA, S.A. (INDITEX)
3702                        ZARA ESPAÑA, S.A. (INDITEX)
3704                                ZARA HOME ESPAÑA SA
6300                       AUTOCARES ZARAPUZ E HIJOS SL
7384                        ZARA ESPAÑA, S.A. (INDITEX)
9004              GESTION HOSTELERA LA ALMAZARA, S.L.U.
10460                       ZARA ESPAÑA, S.A. (INDITEX)
10463                               ZARA HOME ESPAÑA SA
10654                         ZARAGOZA VELILLA ALMUDENA
18286                       ZARA ESPAÑA, S.A. (INDITEX)
19815                       ZARA ESPAÑA, S.A. (INDITEX)
21098                       ZARA ESPAÑA, S.A. (INDITEX)
22796                 PROMOCIONES ZARA DE HUMANE

In [72]:
from scipy.spatial.distance import cdist


In [74]:
points = [(i, j) for i, j in zip(np.arange(40.3, 40.8, 0.02), np.arange(-3.6, -3.0, 0.02))]

In [77]:
np.argmin(cdist([(40.5, -3.2)], points, metric='cityblock'))

10